In [1]:
import yfinance as yf
import numpy as np
from skopt import Optimizer
from skopt.space import Real
from skopt.learning import ExtraTreesRegressor
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import vectorbt as vbt

## Acknowledgement 

Based on the following [article](https://wire.insiderfinance.io/trading-like-a-pro-with-asymmetric-ema-and-bayesian-optimization-a-journey-into-the-future-of-329f834b7bee).

## Data

In [2]:
symbol = 'META' # 'NVDA' # 'MSFT' #'TSLA' # 'AAPL'
start_date = '2023-01-01'
end_date = '2024-12-31'
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)
close_prices = df['Close'].values

[*********************100%***********************]  1 of 1 completed


In [3]:
nrecent = 100
fig = go.Figure(data=[go.Candlestick(
    x=df.index[-nrecent:],
    open=df['Open'][-nrecent:],
    high=df['High'][-nrecent:],
    low=df['Low'][-nrecent:],
    close=df['Close'][-nrecent:]
)])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(
    title=f'{symbol} Stock Price',
    title_x=0.5,  # Center the title
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly_dark'
)
fig.show()

## Asymmetric EMA

In [4]:
def calculate_asy_ema(prices: np.ndarray, alpha: float) -> np.ndarray:
    """Computes the Asymmetric Exponential Moving Average (AsyEma).

    The AsyEma is a variation of the Exponential Moving Average (EMA)
    that applies different weightings to price movements depending on
    whether the current price is increasing or decreasing relative to the previous value.

    The formula used for the AsyEma is:

        AsyEma[i] = alpha * max(price[i], AsyEma[i-1]) + (1 - alpha) * min(price[i], AsyEma[i-1])

    Args:
        prices (np.ndarray): A NumPy array of price values.
        alpha (float): The smoothing factor (0 < alpha ≤ 1),
                       where a higher value gives more weight to recent prices.

    Returns:
        np.ndarray: A NumPy array containing the computed AsyEma values.
    """
    asy_ema = np.zeros_like(prices)
    asy_ema[0] = prices[0]  # Initialize the first value

    for i in range(1, len(prices)):
        asy_ema[i] = alpha * max(prices[i], asy_ema[i - 1]) + (1 - alpha) * min(prices[i], asy_ema[i - 1])

    return asy_ema

## Standard EMA

In [5]:
def calculate_standard_ema(prices: np.ndarray, smoothing_factor: float) -> np.ndarray:
    """Computes the Standard Exponential Moving Average (EMA).

    The Exponential Moving Average (EMA) is a weighted moving average
    that gives more importance to recent prices.

    The formula used for the EMA is:

        EMA[i] = (price[i] - EMA[i-1]) * multiplier + EMA[i-1]

    where:

        multiplier = 2 / (1 + smoothing_factor)

    Args:
        prices (np.ndarray): A NumPy array of price values.
        smoothing_factor (float): The smoothing factor (α),
                                  where a higher value gives more weight to recent prices.

    Returns:
        np.ndarray: A NumPy array containing the computed EMA values.
    """
    ema_values = np.zeros_like(prices)
    ema_values[0] = prices[0]  # Initialize with the first price value

    multiplier = 2 / (1 + smoothing_factor)

    for i in range(1, len(prices)):
        ema_values[i] = (prices[i] - ema_values[i - 1]) * multiplier + ema_values[i - 1]

    return ema_values

In [6]:
asy_ema_1 = calculate_asy_ema(close_prices, 0.1)
asy_ema_2 = calculate_asy_ema(close_prices, 0.2)
asy_ema_3 = calculate_asy_ema(close_prices, 0.3)

ema_1 = calculate_standard_ema(close_prices, 0.1)

In [7]:
fig = go.Figure()

# Close Price - Bold Line
fig.add_trace(go.Scatter(
    x=df.index, y=df['Close'],
    mode='lines', name='Close Price',
    line=dict(width=1, dash='solid', color='white')
))

# AsyEma Lines - Different Line Styles
fig.add_trace(go.Scatter(
    x=df.index, y=asy_ema_1,
    mode='lines', name='AsyEma (α=0.1)',
    line=dict(width=1, dash='dot', color='cyan')
))

fig.add_trace(go.Scatter(
    x=df.index, y=asy_ema_2,
    mode='lines', name='AsyEma (α=0.2)',
    line=dict(width=1, dash='dot', color='magenta')
))

fig.add_trace(go.Scatter(
    x=df.index, y=asy_ema_3,
    mode='lines', name='AsyEma (α=0.3)',
    line=dict(width=1, dash='dot', color='lime')
))

# Standard EMA Lines - Different Line Styles
# fig.add_trace(go.Scatter(
#     x=df.index, y=ema_1,
#     mode='lines', name='Ema (α=0.1)',
#     line=dict(width=1, dash='solid', color='yellow')
# ))

# Layout Enhancements
fig.update_layout(
    title='Comparing EMAs',
    title_x=0.5,
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly_dark',
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(0,0,0,0.5)')
)

fig.show()


## Bayesian Optimisation

In [8]:
def evaluate_agent(alpha: float, delta: float, closing_prices: np.ndarray) -> float:
    """Evaluates the performance of an agent based on a strategy using AsyEma.

    The agent buys shares when the price is sufficiently close to the AsyEma value
    and sells them when the price moves away. The strategy aims to minimize the
    net negative return.

    Args:
        alpha (float): The smoothing factor used in the AsyEma calculation.
        delta (float): The threshold that determines when the price is considered close to the AsyEma.
        closing_prices (np.ndarray): A NumPy array of the closing prices of an asset.

    Returns:
        float: The net negative return, which is used for optimization (minimization).
    """
    try:
        asy_ema = calculate_asy_ema(closing_prices, alpha)
        position = 0  # 0: not in position, 1: long position
        initial_cash = 100000  # Starting capital
        cash = initial_cash
        shares = 0

        for i in range(len(closing_prices)):
            if abs(asy_ema[i] - closing_prices[i]) < delta:
                if position == 0:
                    # Buy: Enter long position
                    shares = cash // closing_prices[i]
                    cash -= shares * closing_prices[i]
                    position = 1
            else:
                if position == 1:
                    # Sell: Exit long position
                    cash += shares * closing_prices[i]
                    shares = 0
                    position = 0

        # Final sale if still holding shares
        cash += shares * closing_prices[-1]

        # Calculate the net return (negative for minimization)
        net_return = -(cash - initial_cash)  # Minimize the negative return

        # Ensure net return is a scalar value
        if isinstance(net_return, np.ndarray):
            net_return = net_return.item()  # Convert to scalar if it's an array
        if not isinstance(net_return, (int, float, np.number)):
            raise ValueError(f"Net return is not a scalar: {net_return}")

        return net_return
    except Exception as e:
        print(f"Error during agent evaluation with alpha={alpha}, delta={delta}: {e}")
        return float('inf')  # Return a high value to signal an error

In [9]:
evaluate_agent(0.01, 0.01, close_prices)

-2106.4883422851562

Run the optimizer

In [10]:
space = [Real(0.01, 0.5, name='alpha'), Real(0.1, 10, name='delta')]

In [11]:
# Initialize optimizer with ExtraTreesRegressor
opt = Optimizer(space, base_estimator=ExtraTreesRegressor(n_estimators=100, random_state=42))

# Run Bayesian optimization
n_calls = 100
objective_values = []
for i in range(n_calls):
    next_x = opt.ask()
    f_val = evaluate_agent(next_x[0], next_x[1], close_prices)
    objective_values.append(-f_val)
    opt.tell(next_x, f_val)

In [12]:
# Extract best parameters and corresponding performance
result = opt.get_result()
best_alpha = result.x[0]
best_delta = result.x[1]
best_performance = -result.fun

In [13]:
print(f"Best alpha: {best_alpha:.4f}")
print(f"Best delta: {best_delta:.4f}")
print(f"Best performance: {best_performance:.2f}")

Best alpha: 0.2142
Best delta: 7.8470
Best performance: 508996.86


In [14]:
asy_ema_opt = calculate_asy_ema(close_prices, best_alpha)

In [15]:
fig = go.Figure()

# Close Price - Bold Line
fig.add_trace(go.Scatter(
    x=df.index, y=df['Close'],
    mode='lines', name='Close Price',
    line=dict(width=1, dash='solid', color='white')
))

fig.add_trace(go.Scatter(
    x=df.index, y=asy_ema_opt,
    mode='lines', name=f'AsyEma (α={best_alpha:.2f})',
    line=dict(width=1, dash='dot', color='lime')
))

# Layout Enhancements
fig.update_layout(
    title='Comparing EMAs',
    title_x=0.5,
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly_dark',
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(0,0,0,0.5)')
)

fig.show()

## Trade Signal

In [16]:
def generate_trade_signals(alpha: float, delta: float, df: pd.DataFrame) -> pd.DataFrame:
    """Generates buy/sell/hold signals based on the AsyEma strategy, ensuring that we sell only if we own shares.

    The function calculates the AsyEma and compares the closing prices to the AsyEma.
    A buy signal (1) is generated when the price is sufficiently close to the AsyEma, a sell signal (-1)
    is generated if the price is far from the AsyEma and we already hold shares, and a hold signal (0)
    is generated otherwise.

    Args:
        alpha (float): The smoothing factor used in the AsyEma calculation.
        delta (float): The threshold that determines when the price is considered close to the AsyEma.
        df (pd.DataFrame): A pandas dataframe of open/close prices of an asset.

    Returns:
        pd.DataFrame: trade signals, where 1 is a buy signal, -1 is a sell signal, and 0 is a hold signal.
    """
    df = df.copy()
    closing_prices = df['Close'].values

    # Calculate AsyEma values based on the given alpha
    asy_ema = calculate_asy_ema(closing_prices, alpha)

    # Initialize an array to hold the trade signals
    signals = np.zeros_like(closing_prices, dtype=int)

    # Track whether the agent owns shares or not
    owns_shares = False

    # Iterate over the closing prices and calculate signals
    for i in range(1, len(closing_prices)):
        if abs(asy_ema[i] - closing_prices[i]) < delta:
            if not owns_shares:
                # Buy signal: price is close to AsyEma and we don't own shares
                signals[i] = 1
                owns_shares = True  # Now we own shares
        elif closing_prices[i] < asy_ema[i] and owns_shares:
            # Sell signal: price is lower than AsyEma and we own shares
            signals[i] = -1
            owns_shares = False  # Now we no longer own shares
        else:
            # Hold signal: Otherwise
            signals[i] = 0
    df['Signal'] = signals
    return df

def signal_to_action(signal: pd.Series) -> pd.DataFrame:
    """
    Convert trading signals into buy/sell action indicators.

    This function takes a signal series and generates a DataFrame with two columns:
    'buy' and 'sell'. A value of 1 in 'buy' indicates a buy action, while a value of -1
    in 'sell' indicates a sell action.

    Args:
        signal (pd.Series): A pandas Series containing trading signals (1 for buy, -1 for sell, 0 for hold).

    Returns:
        pd.DataFrame: A DataFrame with 'buy' and 'sell' columns indicating the respective actions.
    """
    columns = ['buy', 'sell']
    action = pd.DataFrame(np.zeros((signal.shape[0], 2)),
                          index=signal.index,
                          columns=columns,
                          dtype=int)
    action.loc[signal == 1, 'buy'] = 1
    action.loc[signal == -1, 'sell'] = -1
    return action


In [17]:
df = generate_trade_signals(best_alpha, best_delta, df)
df_signal = signal_to_action(df['Signal'])

In [18]:
df_signal.head()

,buy,sell
Date,,
2023-01-03,0,0
2023-01-04,1,0
2023-01-05,0,0
2023-01-06,0,0
2023-01-09,0,0


In [19]:
df_signal.tail()

,buy,sell
Date,,
2024-12-23,0,0
2024-12-24,0,0
2024-12-26,0,0
2024-12-27,0,0
2024-12-30,0,0


## Backtesting

In [20]:
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df_signal['buy'],
    exits=df_signal['sell'],
    init_cash=10_000,
    fees=0.001,
    freq='D',
)

In [21]:
portfolio.stats()

Start                         2023-01-03 00:00:00
End                           2024-12-30 00:00:00
Period                          501 days 00:00:00
Start Value                               10000.0
End Value                            46256.676755
Total Return [%]                       362.566768
Benchmark Return [%]                    375.78844
Max Gross Exposure [%]                      100.0
Total Fees Paid                         79.231728
Max Drawdown [%]                        18.940198
Max Drawdown Duration            61 days 00:00:00
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                       11635.852424
Win Rate [%]                                100.0
Best Trade [%]                         246.454452
Worst Trade [%]                        246.454452
Avg Winning Trade [%]                  246.454452
Avg Losing Trade [%]                          NaN


In [22]:
fig = portfolio.plot_orders()
fig.update_layout(template='plotly_dark', width=1200)
fig.show()

In [23]:
fig = portfolio.plot_trade_pnl()
fig.update_layout(template='plotly_dark', width=1200)
fig.show()